In [1]:
from math import sqrt, fabs

#### Target function

In [2]:
def first_target_function(x):
    return x**2 - 4*x

#### Step and start point

In [3]:
start_point = 3
step_size = 0.1

#### Sven's method

In [4]:
def get_sign(f, x, step):
    left_shift_value = f(x - fabs(step))
    current_value = f(x)
    right_shift_value = f(x + fabs(step))

    if left_shift_value >= current_value >= right_shift_value:
        return 1
    if left_shift_value <= current_value <= right_shift_value:
        return -1
    if left_shift_value >= current_value <= right_shift_value:
        return 0
    else:
        raise Exception("Function isn't unimodal")


def sven_method(f, x0, step):
    x_current = x0
    sign = get_sign(f, x_current, step)
    x_next = x_current + sign * step
    counter = 1

    while True:
        if sign == 0:
            return x_current, x_next
        if f(x_next) > f(x_current):
            return (x_next, x_current) if x_next < x_current else (x_current, x_next)

        x_current = x_next
        sign = get_sign(f, x_current, step)
        x_next = x_current + 2**counter * sign * step
        counter += 1


a, b = sven_method(first_target_function, start_point, step_size)
(a, b)

(1.4999999999999998, 2.3)

#### Accuracy for dichotomy method and golden section

In [5]:
dichotomy_accuracy = 0.2
golden_section_accuracy = 0.01

#### Dichotomy method

In [6]:
def dichotomy_method(f, start, end, epsilon):
    length = end - start
    if fabs(length) < epsilon:
        return (end + start) / 2

    xm = (start + end) / 2
    x1 = start + length / 4
    x2 = end - length / 4

    if f(x1) < f(xm):
        return dichotomy_method(f, start, xm, epsilon)

    if f(x2) < f(xm):
        return dichotomy_method(f, xm, end, epsilon)

    if f(x1) > f(xm):
        return dichotomy_method(f, x1, x2, epsilon)


dichotomy_method(first_target_function, a, b, dichotomy_accuracy)

1.9999999999999998

#### Golden Section

In [7]:

def golden_section(f, start, end, epsilon):
    golden_section_phi = (1 + sqrt(5)) / 2
    res_phi = 2 - golden_section_phi

    _start = start
    _end = end

    x1 = _start + res_phi * (_end - _start)
    x2 = _end - res_phi * (_end - _start)

    while True:
        if f(x1) < f(x2):
            _end = x2
            x2 = x1
            x1 = _start + res_phi * (_end - _start)
        else:
            _start = x1
            x1 = x2
            x2 = _end - res_phi * (_end - _start)

        if fabs(_end - _start) <= epsilon:
            break

    return (x1 + x2) / 2


golden_section(first_target_function, a, b, golden_section_accuracy)

2.001699437494742

#### DSC-Powell's method

In [8]:
powell_start_point = 0
powell_step_size = 0.1
powell_accuracy = 0.01


def dsc_method(f, start, step):
    interval_start, interval_end = sven_method(f, start, step)
    x1 = interval_start
    x2 = (fabs(interval_end) + fabs(interval_start)) / 2
    x3 = interval_end

    xs = x2 + ((x3 - x2) * (f(x1) - f(x3))) / (2 * (f(x1) - 2*f(x2) + f(x3)))
    return interval_start, interval_end, (interval_start + interval_end) / 2, xs


def dsc_powell(f, start, step, epsilon):
    interval_start, interval_end, x_min, xs = dsc_method(f, start, step)

    while fabs(x_min - xs) >= epsilon and fabs(f(x_min) - f(xs)) >= epsilon:
        if xs < x_min:
            interval_end = x_min
            x_min = xs
        elif xs > x_min:
            interval_start = x_min
            x_min = xs
        else:
            break

        f_arr = [f(interval_start), f(x_min), f(interval_end)]
        a1 = (f_arr[1] - f_arr[0]) / (x_min - a)
        a2 = ((f_arr[2] - f_arr[0]) / (interval_end - interval_start) - a1) / (interval_end - x_min)

        xs = (interval_start + x_min) / 2 - a1 / (2 * a2)

        f_min = min(f_arr)
        if f_arr[0] == f_min:
            x_min = interval_start
        elif f_arr[1] == f_min:
            x_min = x_min
        else:
            x_min = interval_end

    return x_min


dsc_powell(first_target_function, powell_start_point, powell_step_size, powell_accuracy)

1.9999999999999998

#### Newton-Rafson

In [9]:
nr_start_point = 0.1
nr_accuracy = 0.01


def second_target_function(x):
    return x**2 + 4/x


def second_derivative_target_function(x):
    return 2*x - 4 / x**2


def newton_rafson(f, df, x0, epsilon):
    point = x0
    delta = fabs(0 - f(point))

    while delta > epsilon:
        point = point - f(point) / df(point)
        delta = fabs(0 - f(point))

    return point


newton_rafson(second_target_function, second_derivative_target_function, nr_start_point, nr_accuracy)

-1.5862336671679858

#### Method of Bolzano

In [10]:
bolzano_x1 = 0.1
bolzano_x2 = 2.0
bolzano_accuracy = 0.01


def bolzano_method(df, x1, x2, epsilon):
    _start = x1
    _end = x2
    middle = (_start + _end) / 2

    while fabs(_end - _start) > epsilon:
        if df(middle) < 0:
            _start = middle
        else:
            _end = middle

        middle = (_start + _end) / 2

    return middle


bolzano_method(second_derivative_target_function, bolzano_x1, bolzano_x2, bolzano_accuracy)

1.2615234375

#### Chord's method

In [11]:
chords_x1 = 0.1
chords_x2 = 2.0
chords_accuracy = 0.01


def chord_method(f, df, start, end, epsilon):
    x0 = (start + end) / 2
    xn = f(x0)
    xn1 = xn - f(xn) / df(xn)
    while fabs(xn1 - xn) > epsilon:
        xn = xn1
        xn1 = xn - f(xn) / df(xn)
    return xn1


chord_method(second_target_function, second_derivative_target_function, chords_x1, chords_x2, chords_accuracy)

-1.5874010519712742